<a href="https://colab.research.google.com/github/kanishk16/Stepping-Up-with-CV/blob/master/Ball_Tracking_(Day_05).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# neccessary imports
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from imutils.video import VideoStream 
# from imutils import contours
# from imutils.perspective import four_point_transform
from collections import deque
import imutils
import time
from google.colab.patches import cv2_imshow as cv_imshow

# version check
cv.__version__

'4.1.2'

> STEPS to BALL TRACKin

* Detect the presence of a ball in the video
* Track the ball as it moves in the video frame (discretization of the path)

NOTE: For more ideas n applns. read the discussions of this [blog](https://www.pyimagesearch.com/2015/09/14/ball-tracking-with-opencv/?__s=pp3fsqvmd1wahefsfr5s)

In [2]:
# %%time
## Detecting the ball in the video frame
# define the green band(lower + upper) -- range-detector module: imutils/bin/range-detector 
low_green = (29, 86, 6)
high_green = (64, 255, 255)

# initializing the list of track pts.
pts = deque(maxlen = 20)

# grabbing ref. to the video file
vs = cv.VideoCapture('/content/ball_tracking_example.mp4')

print("FPS:",vs.get(cv.CAP_PROP_FPS))

trackinball_frames = []

frame_count = 0

# loop on..
while True:

  # grab the current frame 
  frame = vs.read()

  # count the num of frames
  frame_count += 1
  
  # vs.read() - returns a tuple (True/False, ndarray) 
  # cv_imshow(frame[1])

  # if not frame:
  #   print("No No")
  #   break

  # exit if no frame returned
  if frame[1] is None:
    break

  # resize = process the frame faster + inc. in FPS(less img. data to process) 
  framer = imutils.resize(frame[1], width=600)
  
  # blur = reduce high freq. noise + focus strucutral obj. inside the fame
  blur_fr = cv.GaussianBlur(framer, (11,11), 0)
  
  # conversion to HSV = identification specifications
  hsv_fr = cv.cvtColor(blur_fr, cv.COLOR_BGR2HSV)

  # construct a mask + localization of the GREEN ball
  mask = cv.inRange(hsv_fr, low_green, high_green) 
  mask_fr = cv.dilate(mask, None, iterations=2)    
  mask_fr = cv.erode(mask, None, iterations=2)

  # cv_imshow(freeze)
  # cv_imshow(mask_fr)

  # find the contours of the ball from the mask frame
  mask_contours = cv.findContours(mask_fr.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[0]

  # initialize the center
  center = None
  # print(len(mask_contours))
  
  # apply only if any contour is found    
  # if mask_contours is None:
  #   continue
  
  if len(mask_contours)>0:

    # find the largest contour in the mask
    # print(type(mask_contours))
    cntrs = max(mask_contours, key=lambda x: cv.contourArea(x))

    # compute the min. enclosing circle 
    ((x,y), r) = cv.minEnclosingCircle(cntrs)

    # compute the centroid 
    k = cv.moments(cntrs)
    center = (int(k["m10"] / k["m00"]), int(k["m01"] / k["m00"]))

    # proceed only if radius meets the specific size
    if r>10:
      # draw the circle on the frame
      cv.circle(framer, (int(x),int(y)), int(r), (0,255,255), 2)
      # draw the centroid on the frame
      cv.circle(framer, center, 5, (0,0,255), -1)

  # update the pts[] with the center of the ball in this particular frame
  pts.appendleft(center)

  # draw the contrail of the ball
  for i in range(1, len(pts)):

    # ignore the pts if None
    if pts[i-1] is None or pts[i] is None:
      continue

    # compute the thickness of the line + draw the connecting lines
    thickness = int(np.sqrt(20/float(i+1)*2.5))
    cv.line(framer, pts[i-1], pts[i], (0,0,255), thickness)

  trackinball_frames.append(framer)
  # cv_imshow(framer)

  # key = cv.waitKey(1) & 0xFF

  # STOP as soon as the 'q' key is pressed
  # if key == ord("q"):
  #   break
print("Frames:", frame_count, "Frame Arr:", trackinball_frames, "Frame arr len:", len(trackinball_frames))

FPS: 30.0
Frames: 1138 Frame Arr: [array([[[253, 253, 253],
        [253, 253, 253],
        [253, 253, 253],
        ...,
        [ 85, 100,  98],
        [ 85, 100,  98],
        [ 85, 100,  98]],

       [[253, 253, 253],
        [253, 253, 253],
        [253, 253, 253],
        ...,
        [ 86, 101,  99],
        [ 86, 101,  99],
        [ 86, 101,  99]],

       [[253, 253, 253],
        [253, 253, 253],
        [253, 253, 253],
        ...,
        [ 87, 102, 100],
        [ 87, 102, 100],
        [ 87, 102, 100]],

       ...,

       [[253, 253, 253],
        [253, 253, 253],
        [253, 253, 253],
        ...,
        [101, 112, 120],
        [ 99, 110, 118],
        [100, 110, 118]],

       [[253, 253, 253],
        [253, 253, 253],
        [253, 253, 253],
        ...,
        [101, 109, 118],
        [101, 109, 118],
        [101, 109, 118]],

       [[253, 253, 253],
        [253, 253, 253],
        [253, 253, 253],
        ...,
        [ 99, 110, 118],
        [ 99, 

> Applications 

* Trace multiple objects simulatanously of SAME colour (headlights)+ speed (Sports)
    * Creating a different mask for diff. colors  
* Tracking a squash ball (Challenges)-
    * black ball
    * diameter of ball being small + percieved diameter of the ball even smaller
    * speed of the ball

* Dispatching actions based on the ball reaching the bounding box in the frame(top-left) 
* Tracking the [tracking direction](https://www.pyimagesearch.com/2015/09/21/opencv-track-object-movement/)(left to right, south...)


In [0]:
frame_shape = (600, 333)

# create a video writer object
vw = cv.VideoWriter("ball_tracking_processed.mp4", cv.VideoWriter_fourcc(*'DIVX'), 30, frame_shape)

for i in range(len(trackinball_frames)):
  # write the frames
  vw.write(trackinball_frames[i])

vw.release()  